In [228]:
import pandas as pd
import glob
import json
import os
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

#Folder path must end with /
FOLDER = "/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdp_api_results_2019.json',
 'basic1_tmdb_data_2013.csv.gz',
 'tmdp_api_results_2023.json',
 'tmdp_api_results_2015.json',
 '.DS_Store',
 'tmdp_api_results_2014.json',
 'tmdp_api_results_2022.json',
 'tmdp_api_results_2018.json',
 'basic1_tmdb_data_2011.csv.gz',
 'tmdp_api_results_2013.json',
 'basic1_tmdb_data_2015.csv.gz',
 'basic1_tmdb_data_2019.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'basic1_tmdb_data_2023.csv.gz',
 'title.basics.tsv.gz',
 'basic1_tmdb_data_2017.csv.gz',
 'basic1_tmdb_data_2021.csv.gz',
 'title_basics.csv.gz',
 'tmdp_api_results_2012.json',
 'tmdp_api_results_2011.json',
 'tmdp_api_results 2001.json',
 'basic1_tmdb_data_2012.csv.gz',
 'all_movies.csv',
 'title.ratings.tsv.gz',
 'basic1_tmdb_data_2010.csv.gz',
 'title-akas-us-only.csv',
 'tmdp_api_results 2000.json',
 'tmdp_api_results_2010.json',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'tmdp_api_results_2021.json',
 'basic1_tmdb_data_2022.csv.gz',
 'basic1_tmdb_data_2014.csv.gz',
 'basic1_tmdb_

In [168]:
#Loading in API keys
with open("/Users/echo/Documents/0424_Data_Enrichment/.secret/tmdb_api.json", "r") as f:    
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()    
tmdb.API_KEY =  login['api-key']

In [169]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv("/Users/echo/Documents/0424_Data_Enrichment/Project-3/Data/basics_input_part4.csv")
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
1995,tt0264365,movie,Ancient Warriors,Ancient Warriors,0,2003.0,NaN,99,Action
1996,tt0264395,movie,Basic,Basic,0,2003.0,NaN,98,"Action,Crime,Drama"
1997,tt0264410,movie,Berserker,Berserker,0,2004.0,NaN,84,"Action,Adventure,Fantasy"
1998,tt0264425,movie,Bill's Gun Shop,Bill's Gun Shop,0,2001.0,NaN,93,"Crime,Drama"


In [170]:
def get_movie_with_rating(movie_id):
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    #Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    #Loop thru countries in releases
    for c in releases["countries"]:
        #if the country abbreviation == US
        if c["iso_3166_1"] == "US":
            #Save a certification key to new column
            info["certification"] = c["certification"]
    return info

In [171]:
#Example of what info the custom function returns
movie = get_movie_with_rating("tt0378194")
movie.keys()

dict_keys(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'certification'])

# Create Required Lists for the Loop
## Define a list of the Years to Extract from the API

In [172]:
YEARS_TO_GET = range(2010,2024)

## Define an errors list

In [173]:
#Create empty list to append errors later on 
errors = []

In [174]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## Start OUTER loop

In [175]:
# Start of OUTER loop
#progress bar based on years_to_get. DESC means Description name
for YEAR in tqdm_notebook(YEARS_TO_GET, desc="YEARS", position=0):
    #Defining the JSON file to store results for each year
    JSON_FILE = f"{FOLDER}tmdp_api_results_{YEAR}.json"
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict to the new json file.
        with open(JSON_FILE, "w") as f:
            json.dump([],f)
    #Saving new year as the current df
#     df = basics.loc[basics["startYear"] == YEAR].copy()
    # saving movie ids to list
    movie_ids = basics["tconst"].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    try:
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]
        movie_ids_to_get
    except:
        pass
    
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

                
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}basic1_tmdb_data_{YEAR}.csv.gz",
                        compression="gzip", index=False)
print(f"- Total errors: {len(errors)}")   

YEARS:   0%|          | 0/14 [00:00<?, ?it/s]

Movies from 2010:   0%|          | 0/2000 [00:00<?, ?it/s]

Movies from 2011:   0%|          | 0/2000 [00:00<?, ?it/s]

Movies from 2012:   0%|          | 0/2000 [00:00<?, ?it/s]

Movies from 2013:   0%|          | 0/2000 [00:00<?, ?it/s]

...

Movies from 2020:   0%|          | 0/2000 [00:00<?, ?it/s]

Movies from 2021:   0%|          | 0/2000 [00:00<?, ?it/s]

Movies from 2022:   0%|          | 0/2000 [00:00<?, ?it/s]

Movies from 2023:   0%|          | 0/2000 [00:00<?, ?it/s]

- Total errors: 28000


# Using glob to get a list of files that match a pattern

In [176]:
#Looking for any files with this pattern and saving to variable
q = "Data/basic1_tmdb_data_20*.csv.gz"
chunked_files = glob.glob(q)
# Showing the first 5
sorted(chunked_files)

['Data/basic1_tmdb_data_2010.csv.gz',
 'Data/basic1_tmdb_data_2011.csv.gz',
 'Data/basic1_tmdb_data_2012.csv.gz',
 'Data/basic1_tmdb_data_2013.csv.gz',
 'Data/basic1_tmdb_data_2014.csv.gz',
 'Data/basic1_tmdb_data_2015.csv.gz',
 'Data/basic1_tmdb_data_2016.csv.gz',
 'Data/basic1_tmdb_data_2017.csv.gz',
 'Data/basic1_tmdb_data_2018.csv.gz',
 'Data/basic1_tmdb_data_2019.csv.gz',
 'Data/basic1_tmdb_data_2020.csv.gz',
 'Data/basic1_tmdb_data_2021.csv.gz',
 'Data/basic1_tmdb_data_2022.csv.gz',
 'Data/basic1_tmdb_data_2023.csv.gz']

In [241]:
## Loading all files as df and appending to a list
df_list = []
for file in chunked_files:
    print(file)
    temp_df = pd.read_csv(f"{file}")
    df_list.append(temp_df)

# Concatenating the list of dfs into 1 combined
df_combined = pd.concat(df_list)


Data/basic1_tmdb_data_2013.csv.gz
Data/basic1_tmdb_data_2011.csv.gz
Data/basic1_tmdb_data_2015.csv.gz
Data/basic1_tmdb_data_2019.csv.gz
Data/basic1_tmdb_data_2023.csv.gz
Data/basic1_tmdb_data_2017.csv.gz
Data/basic1_tmdb_data_2021.csv.gz
Data/basic1_tmdb_data_2012.csv.gz
Data/basic1_tmdb_data_2010.csv.gz
Data/basic1_tmdb_data_2022.csv.gz
Data/basic1_tmdb_data_2014.csv.gz
Data/basic1_tmdb_data_2018.csv.gz
Data/basic1_tmdb_data_2020.csv.gz
Data/basic1_tmdb_data_2016.csv.gz


In [242]:
#List comprehension to combine files into 1 df
## Loading and Concatenating the list of dfs with 1 line
df = pd.concat([pd.read_csv(file) for file in chunked_files])
df.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232,tt0035423,en,Kate & Leopold,...,76019048,118,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,False,6.330,1216,PG-13
1,False,/fw5tsNib4QZBEw18xmebpVe3WZ8.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",http://poetastros.com/el-tango-del-viudo/,602986,tt0062336,es,El tango del viudo y su espejo deformante,...,0,63,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,NaN,The Tango of the Widower and Its Distorting Mi...,False,6.167,6,NaN
2,False,/7kIfosP2TlzZeD4a8b2He9COnwg.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,124023,tt0068865,en,Lives of Performers,...,0,90,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Lives of Performers,False,5.300,3,NaN
3,False,/wXUcSJG6dqNKgIRgqYqX98UA1wz.jpg,NaN,12000000,"[{'id': 18, 'name': 'Drama'}]",https://www.netflix.com/title/80085566,299782,tt0069049,en,The Other Side of the Wind,...,0,122,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,40 years in the making.,The Other Side of the Wind,False,6.700,185,R
4,False,NaN,NaN,350000,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,29163,tt0088751,en,The Naked Monster,...,0,100,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Naked Monster,False,4.000,6,NaN


3 Question to Answer:

Q1. Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates? (ANOVA)

Q2. Is there a significant difference in revenue between movies that belong to a collection and those that do not? (T-Test Independent)

Q3. Does the release month have an effect on revenue? (ANOVA)

## Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?

1.State your Null Hypothesis and Alternative Hypothesis

   * Null Hypothesis: There is no signifcant difference in revenue between genres.
    
   * Alternative Hypothesis: There is a significant difference revenue between genres.

2.Select the correct test according to the data type and number of samples

   * ANOVA 

3.Test the assumptions of your selected test.

   *  No significant outliers
   *  Normality
   *  Equal Variance

4.Execute the selected test, or the alternative test (if you do not meet the assumptions)
   
   * I'll be looking out for the p-value from the .levene variance test to determine whether to use the nonparametric tests (medians), Kruskal-Wallis.

5.Interpret your p-value and reject or fail to reject your null hypothesis 
 
    
6.Show a supporting visualization that helps display the result


In [243]:
#Checking the size of the groups
df["certification"].value_counts(dropna=False)

NaN        15563
R           7259
PG-13       3227
PG          1165
NR           662
G            293
NC-17         89
Unrated       14
Name: certification, dtype: int64

In [244]:
#Dropping unnecessary genres
df = df.drop(df[df["certification"] == "Unrated"].index)
df = df.drop(df[df["certification"] == "NC-17"].index)
df = df.drop(df[df["certification"] == "NR"].index)

In [245]:
#Dropping NaN values in certification columm
df = df.dropna(subset=["certification"])

In [246]:
## Create groups dictionary. There are only 3 groups, but this is even more helpful when there are many groups
genre_group = {}
## Loop through all unique categories
for i in df["certification"].unique():
    ## Get series for group and rename
    data = df.loc[df["certification"] == i, "revenue"].copy()
    
    # save into the dictionary
    genre_group[i] = data
genre_group.keys()    

dict_keys(['PG-13', 'R', 'PG', 'G'])

## Testing Assumptions


### Normality Test

In [247]:
## Running normal test on each group and confirming there are >20 in each group
norm_results = {}
for i, data in genre_group.items():
    stat, p = stats.normaltest(data)
    ## save the p val, test statistic, and the size of the group
    norm_results[i] = {"n": len(data),
                      "p":p,
                      "test stat":stat}
## convert to a dataframe
norm_results_df = pd.DataFrame(norm_results).T
norm_results_df

,n,p,test stat
PG-13,3087.0,0.000000e+00,1912.925834
R,6914.0,0.000000e+00,7555.132363
PG,1120.0,9.420822e-135,617.212130
G,281.0,4.263729e-26,116.834136


In [248]:
### checking sig with pandas 
norm_results_df["sig"] = norm_results_df["p"] < .05
norm_results_df

,n,p,test stat,sig
PG-13,3087.0,0.000000e+00,1912.925834,True
R,6914.0,0.000000e+00,7555.132363,True
PG,1120.0,9.420822e-135,617.212130,True
G,281.0,4.263729e-26,116.834136,True


These groups do not have a normal distribution. However, "If your group sizes (n) are large enough, we can safely ignore the normality assumption."

### Testing Assumption of Unequal Variance

In [249]:
# with the * operator 
stats.levene(*genre_group.values())

LeveneResult(statistic=374.2676767736281, pvalue=7.375343321584666e-232)

Levene pvalue < .05, so this is showing it is NOT of equal variance. Meaning that we reject the null hypothesis.

### Checking for Outliers

In [250]:
## Loop through groups dict
for i, data in genre_group.items():

    ## determine if there are any outliers
    outliers = np.abs(stats.zscore(data)) > 3
    
    ## print a statement about how many outliers for which group name
    print(f"There were {outliers.sum()} outliers in the {i} group.")

    ## Remove the outiers from data and overwrite the sector data in the dict
    data = data.loc[~outliers]
    genre_group[i] = data

There were 89 outliers in the PG-13 group.
There were 130 outliers in the R group.
There were 29 outliers in the PG group.
There were 15 outliers in the G group.


### Kruskal Test

In [251]:
# Perform Kruskal-Wallis test
h_statistic, p_value = stats.kruskal(*genre_group.values())

print(f"Kruskal Result: statistic={h_statistic}, pvalue={p_value}")
# Interpret the result
print("Revenue significantly differ among the genres.")

Kruskal Result: statistic=1512.2995693364735, pvalue=0.0
Revenue significantly differ among the genres.


## Post-hoc multiple comparison tests

In [252]:
## save the values of revenu and the labels to the certifications
values = df['revenue']
labels = df['certification']

In [253]:
## perform tukey's multiple comparison test and display the summary
tukeys_results = pairwise_tukeyhsd(values,labels)
tukeys_results.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
G,PG,-15772750.5325,0.2369,-37335658.0467,5790156.9816,False
G,PG-13,-13419595.4326,0.3173,-33557533.0197,6718342.1545,False
G,R,-94809465.5555,0.0,-114476911.9616,-75142019.1494,True
PG,PG-13,2353155.0999,0.9502,-8920360.0744,13626670.2743,False
PG,R,-79036715.0229,0.0,-89446521.0855,-68626908.9603,True
PG-13,R,-81389870.1229,0.0,-88385698.3124,-74394041.9334,True


The test shows that there is a significant difference in the results with the R-rated genre.  The revenue was significantly higher for those R-rated movies.

Therefore, I reject the null hypothesis and choose the alternative hypothesis: There is a significant difference revenue between genres.

# Is there a significant difference in revenue between movies that belong to a collection and those that do not?